## Model Data using Cassandra


### Please just submit this notebook in the Submission and make sure each cell has been executed and output is clearly displayed.

### The aim of the project is to solve the three queries given below.

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.

#### Import Packages 

In [1]:
#! pip install cassandra-driver

import cassandra
import pandas as pd
import numpy as np
import cassandra
import csv
print("imported")

imported



##  The image below is a screenshot of what the data appears like in the event_data.csv

<img src="event_data_image.jpg">

#### Creating a Cluster

In [2]:
# Task: Make a connection to the cassandra instance on your local machine(127.0.0.1) and 
# create a session to establish connection and begin executing queries
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
    print("connected to the local server")
except Exception as e:
    print(e)

connected to the local server


In [3]:
print("Existing Clusters in our DataBase are :")
cluster.metadata.keyspaces

Existing Clusters in our DataBase are :


{'vehicle_tracker': <cassandra.metadata.KeyspaceMetadata at 0x2a1028840d0>,
 'system_auth': <cassandra.metadata.KeyspaceMetadata at 0x2a102884100>,
 'system_schema': <cassandra.metadata.KeyspaceMetadata at 0x2a1028842e0>,
 'home_secuirty': <cassandra.metadata.KeyspaceMetadata at 0x2a102884a60>,
 'nosqldb': <cassandra.metadata.KeyspaceMetadata at 0x2a1028aee30>,
 'system_distributed': <cassandra.metadata.KeyspaceMetadata at 0x2a1028af400>,
 'system': <cassandra.metadata.KeyspaceMetadata at 0x2a1028af7c0>,
 'system_traces': <cassandra.metadata.KeyspaceMetadata at 0x2a1028bc580>}

In [4]:
### As per the Above list, we dont have the dedicated Keyspace "SoundCLoud".
### So, we will create a new Keysapce named-- SoundCloud

#### Create & Set Keyspace

In [5]:
# Task: Create a Keyspace and Set KEYSPACE to the keyspace specified above

query = """
    CREATE KEYSPACE IF NOT EXISTS SoundCloud 
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""" 
try:
    session.execute(query)
except Exception as e:
    print(e) 
   
try:
    session.execute('USE SoundCloud')
    
except Exception as e:
    print(e)
    
print("KEYSPACE = SOUNDCLOUD \n 1. Created \n 2. SET")  

KEYSPACE = SOUNDCLOUD 
 1. Created 
 2. SET


In [6]:
cluster.metadata.keyspaces

{'vehicle_tracker': <cassandra.metadata.KeyspaceMetadata at 0x2a1028840d0>,
 'system_auth': <cassandra.metadata.KeyspaceMetadata at 0x2a102884100>,
 'system_schema': <cassandra.metadata.KeyspaceMetadata at 0x2a1028842e0>,
 'home_secuirty': <cassandra.metadata.KeyspaceMetadata at 0x2a102884a60>,
 'nosqldb': <cassandra.metadata.KeyspaceMetadata at 0x2a1028aee30>,
 'system_distributed': <cassandra.metadata.KeyspaceMetadata at 0x2a1028af400>,
 'system': <cassandra.metadata.KeyspaceMetadata at 0x2a1028af7c0>,
 'system_traces': <cassandra.metadata.KeyspaceMetadata at 0x2a1028bc580>,
 'soundcloud': <cassandra.metadata.KeyspaceMetadata at 0x2a102884070>}

## List of Queries 

### 1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4


### 2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182
    

### 3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'




### Query1 Table1: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [7]:
## Task: Query 1: Find the artist_name, song_title and length of song the SoundCloud app history 
## that was heard during session_number = 338, and item_in_session_number = 4
## make use of create table command   

In [8]:
query1 = "CREATE TABLE IF NOT EXISTS my_music_library (song_title text, artist_name text, length_of_song text,session_number int ,item_in_session_number int, PRIMARY KEY (session_number,item_in_session_number ))"

try:
                 session.execute(query1)
except Exception as e:
    print(e)

### Let's insert our data into of table

In [9]:
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) 
    for row in csv_reader:
        query2 = "INSERT INTO my_music_library (song_title, artist_name, length_of_song, session_number, item_in_session_number)"
        query2 = query2 + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query2,(row[9],row[0],row[5],int(row[8]),int(row[3])))    

In [10]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query3 = 'SELECT * from my_music_library'

try:
    rows = session.execute(query3)
   
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name,row.item_in_session_number, row.length_of_song, row.session_number ,row.song_title )

Regina Spektor 0 191.08526 23 The Calculation (Album Version)
Octopus Project 1 250.95791 23 All Of The Champs That Ever Lived
Tegan And Sara 2 180.06159 23 So Jealous
Dragonette 3 153.39057 23 Okay Dolores
Lil Wayne / Eminem 4 229.58975 23 Drop The World
Soulja Boy Tell'em 5 201.11628 23 Let Me Get Em
Bodo Wartke 6 645.27628 23 Liebeslied (Sprachen: Deutsch_ Englisch_ FranzÃÂ¶sisch_ Italienisch_ Spanisch_ HollÃÂ¤ndisch_ Japanisch_ Russisch_ Griechisch_ Klingonisch_ Hessisch)
Evanescence 7 237.11302 23 Bring Me To Life
Van Halen 8 243.17342 23 Good Enough
The Academy Is... 9 209.76281 23 Paper Chase (Album Version)
Dwight Yoakam 10 239.3073 23 You're The One
The Far East Movement featuring Wiz Khalifa and Bionik 11 265.50812 23 Lowridin
Amon Amarth 12 238.52363 23 The Hero
Modest Mouse 13 259.47383 23 The Good Times Are Killing Me
Bad Company 14 246.96118 23 Morning Sun
Bodyrox 16 490.78812 23 Yeah Yeah
Alberto Plaza 17 291.5522 23 Yo Te Seguire (vivo)
matchbox twenty 18 225.95873 23

Pet Shop Boys 27 299.02322 140 It's A Sin (2001 Digital Remaster)
Paramore 29 267.65016 140 The Only Exception (Album Version)
Carla Bruni 30 153.20771 140 Promises Like Pie-Crust (Album)
Skillet 31 232.46322 140 Comatose (Comes Alive Version)
Violeta Parra 32 123.34975 140 Parabienes Al Reves
Jonathan Coulton 33 208.03873 140 Chiron Beta Prime
Nirvana 34 219.08853 140 Come As You Are
Metallica 35 385.43628 140 Broken_ Beat & Scarred
Bell Biv DeVoe 36 262.00771 140 Poison
Coldplay 37 273.47546 140 Trouble
Metric 38 172.69506 140 The List
Everything But The Girl 39 259.082 140 Better Things
Calle 13 40 290.84689 140 Un Beso de Desayuno
Justin Bieber 41 214.67383 140 One Time
Ashanti 42 232.25424 140 Foolish
The Darkness 43 217.05098 140 I Believe In A Thing Called Love
Muse 44 209.34485 140 Supermassive Black Hole (Album Version)
Choeur ArmÃÂ©nien de Sofia 45 121.5473 140 Le Corps de Notre Seigneur
Gyptian 46 199.13098 140 Beautiful Lady
Alliance Ethnik 47 252.21179 140 ReprÃÂ©sente
J

The Serendipity Singers 108 163.21261 324 Don't Let The Rain Come Down (Crooked Little Man)
Usher featuring will.i.am 109 395.72853 324 OMG
Los Lobos 110 174.18404 324 La Bamba
Descendents 111 89.23383 324 I'm Not A Loser
Michael Cera & Ellen Page 0 116.71465 439 Anyone Else But You
The Cat Empire 1 218.22649 439 How To Explain
Bryn Terfel / Berliner Philharmoniker / Claudio Abbado 2 967.36608 439 Wotan's Farewell & Magic Fire Music
The Fugees 3 281.20771 439 Ready Or Not
Hardline 4 234.73587 439 Everything
The Funky Lowlives 5 280.34567 439 Sail Into the Sun
DL Incognito 6 221.07383 439 Proof
Justice 7 243.40853 439 TTHHEE PPAARRTTYY
Strawbs 8 255.81669 439 Sheep
Angus & Julia Stone 9 172.85179 439 Wasted
Sara Bareilles 10 260.8322 439 Love Song
Bruna Caram 11 198.63465 439 Meus Sonhos
Nando Reis 12 239.82975 439 O Segundo Sol
The Black Keys 13 189.28281 439 Same Old Thing
Kreator 14 294.53016 439 Riot Of Violence
Audioslave 15 277.83791 439 Show Me How To Live
Red Hot Chili Peppers 1

Slim Cessna's Auto Club 3 333.37424 313 Unto The Day
Kristian Stanfill 0 287.50322 838 I Need You
Enrique Iglesias 1 241.42322 838 Tired Of Being Sorry
Michael Cretu 2 301.06077 838 The Invisible Man
Tommy Emmanuel 3 168.14975 838 Windy & Warm
Faith No More 1 186.14812 959 Easy
Virus Syndicate 2 219.01016 959 Vibrator
The Used 3 190.82404 959 Let It Bleed (Amended Album Version)
A Day To Remember 4 243.22567 959 If It Means A Lot To You
Circle Of Dead Children 5 30.82404 959 Grabbing N
Submersed 6 214.41261 959 Deny Me
Elliott Smith 7 270.62812 959 Twilight
Blindside 8 192.20853 959 Pitiful (LP Version)
Kid Rock 1 296.95955 818 All Summer Long (Album Version)
Ray LaMontagne 2 235.15383 818 Barfly
CSS 3 191.42485 818 Meeting Paris Hilton (Album)
Jason Mraz 4 252.39465 818 Live High (Album Version)
Elena 5 269.58322 818 Setanta matins
Bullet For My Valentine 6 258.2722 818 Her Voice Resides
Deepest Blue 7 210.54649 818 Deepest Blue
Laurie Berkner 8 132.67546 818 The Crabs
Cake 9 190.3277

Coldplay 41 307.51302 758 Clocks
Rihanna 42 268.25098 758 Don't Stop The Music
Tears For Fears 43 255.34649 758 Head Over Heels
Train 44 234.50077 758 Skyscraper
Confederate Railroad 45 202.57914 758 Jesus And Mama (LP Version)
Panic At The Disco 46 259.29098 758 Behind The Sea [Live In Chicago]
Gorillaz 47 221.20444 758 Feel Good Inc (Album Version)
Paramore 48 267.65016 758 The Only Exception (Album Version)
Orishas 50 247.43138 758 Quien Te Dijo
Ricardo Montaner 51 249.41669 758 Y Como  Es El
Loudon Wainwright III 52 147.74812 758 I Suppose
Seeed 53 192.80934 758 Ding (Thing Feat. Saian Supa Crew)
Yung Berg 54 230.63465 758 Do That There (featuring Dude 'N Nem)
Country Joe & The Fish 55 294.76526 758 Rockin Round The World
Emiliana Torrini 56 257.67138 758 Me And Armini
Viola 57 260.25751 758 Nostalgia Amnesia
Panic At The Disco 58 114.31138 758 Folkin' Around [Live In Chicago]
Travis 59 204.7473 758 Good Feeling
Kings Of Leon 60 201.79546 758 Revelry
The Darkness 61 206.81098 758 O

...

Kings Of Leon 2 231.81016 191 Use Somebody
Miike Snow 1 220.83873 435 Black & Blue
Cartola 2 208.92689 435 Sala De RecepÃÂ§ÃÂ£o
Kill The Client 3 70.68689 435 Commander In Thief
Wolfmother 5 175.82975 435 Woman
Old Crow Medicine Show 6 231.73179 435 Wagon Wheel
Architecture In Helsinki 7 173.73995 435 Debbie
Charlie Louvin 8 170.86649 435 I Think I'll Live
Miguel Morales 9 270.78485 435 La Derrota de Un Don Juan
Dominique A 10 153.20771 435 Le Courage Des Oiseaux
Cock Sparrer 11 203.25832 435 Run With The Blind
Jimmy Wakely 12 165.74649 435 I Love You So Much It Hurts
Peter Doherty 13 217.02485 435 A Little Death Around the Eyes
Katy Perry 14 246.41261 435 Thinking Of You
Arctic Monkeys 1 152.60689 329 Baby I'm Yours
Amy Winehouse 1 202.73587 272 You Know I'm No Good
Eddie Floyd 1 198.19057 31 Guess Who
Josh Groban 1 260.70159 531 Little Drummer Boy [featuring guitarist Andy McKee] (Album Version)
Creedence Clearwater Revival 2 271.882 531 I Put A Spell On You
AFI 0 176.48281 166 Mis

ILS 19 350.61506 559 No Soul (High Contrast Remix)
Pearl Jam 20 161.48853 559 Oceans
Armin van Buuren 21 452.23138 559 Never Say Never
Righteous Brothers 22 215.90159 559 Unchained Melody
Misfits 23 45.47873 559 Kong Unleashed (Album Version)
Kid Cudi / MGMT / Ratatat 24 295.67955 559 Pursuit Of Happiness (nightmare)
Owl City 25 267.65016 559 The Technicolor Phase
David Gray 26 304.77016 559 Disappearing World (Album Version)
Tub Ring 27 233.69098 559 Invalid
Frightened Rabbit 28 278.72608 559 Yes_ I Would
Creedence Clearwater Revival 29 315.42812 559 Born On The Bayou
Atheist 30 241.05751 559 On They Slay
T.I. 31 214.77832 559 Why You Wanna (Amended Album Version)
Peggy Zina 32 287.00689 559 Ftes
Bon Iver 33 329.24689 559 Blindsided
The Whitest Boy Alive 34 191.29424 559 Fireworks (bonus track)
Bright Eyes 35 279.87546 559 Poison Oak (Album Version)
Murder By Death 36 182.25587 559 Ball & Chain
Smash Mouth 37 206.36689 559 Walkin' On The Sun
Lauryn Hill 38 280.81587 559 Tell Him
Jes 3

Freddie Bruno 2 230.21669 807 Updated & Still Hated (The Ball Point Composer Album Version)
Coldplay 3 294.1122 807 Fix You
Katrina & The Waves 4 220.9171 807 Walking On Sunshine
Paul Van Dyk Featuring Jessica Sutta 0 425.66485 672 White Lies (Dave Spoon Remix)
Tim Hughes 1 323.47383 672 God of Justice
David Cassidy & The Partridge Family 2 227.73506 672 I'll Meet You Halfway
Snow Patrol 3 200.93342 672 Crack The Shutters
Third Eye Blind 4 347.11465 672 Darwin
HYPOCRISY 5 331.38893 672 The Quest
Peccatum 6 283.0624 672 And Pray For Me
Justyna Steczkowska 7 333.53098 672 Noc...
Bloc Party 8 259.36934 672 Your Visits Are Getting Shorter
Temprees 9 177.34485 672 At Last
Mickie Krause 10 204.17261 672 Orange TrÃÂ¤gt Nur Die MÃÂ¼llabfuhr (Go West)
Foolish Things 11 266.86649 672 Who Can Compare
Bersuit Vergarabat 12 289.30567 672 Hecho En Buenos Aires
ManÃÂ¡ 13 262.81751 672 Mariposa traicionera
Newton Faulkner 14 190.04036 672 Feels Like Home
BjÃÂ¶rk 15 260.25751 672 Come To Me
Olivia 

SOJA 8 355.89179 1075 I Dont Wanna Wait
Infected Mushroom 9 463.72526 1075 Echonomix
Unkle Bob 10 160.10404 1075 One By One
Dwight Yoakam 0 239.3073 325 You're The One
The Devil Wears Prada 1 205.66159 325 Lord Xenu
Veracocha 0 506.93179 715 Carte Blanche
Holy Fuck 0 209.76281 540 Milkshake
Cash Cash 1 212.16608 540 Electric Hearts
OutKast 2 140.17261 540 God
Tab Benoit 3 221.09995 540 Jambalaya
August Burns Red 4 252.21179 540 Composure
Cartola 0 172.12036 133 As Rosas NÃÂ£o Falam
Vampire Weekend 1 269.16526 133 I Think Ur A Contra
Sebadoh 2 210.18077 133 Hassle
Incubus 3 212.74077 133 Wish You Were Here
Blue States 4 239.3073 133 Allies
Devin Townsend 0 576.33914 450 Earth Day
Talking Heads 6 259.02975 450 Road To Nowhere (Remastered LP Version )
Guns N' Roses 7 406.17751 450 Paradise City
Guns N' Roses 0 406.17751 811 Paradise City
A3Bandas 1 107.10159 811 El arca de Noe
Deadmau5 6 429.94893 933 Arguru
Metric 7 286.24934 933 Help I'm Alive
Warren Barfield 8 222.98077 933 Love Is No

Mark Lowry 10 168.28036 350 Get Together With The Lord (The Best Of Mark Lowry - Volume 2 Version)
Beirut 11 230.19057 350 Nantes
MODESELEKTOR FEAT. PUPPETMASTAZ 12 52.79302 350 THE DARK SIDE OF THE FROG
Kid Cudi / Kanye West / Common 13 237.76608 350 Make Her Say
Julie Ruin 14 142.47138 350 Breakout A-Town
Sons And Daughters 15 165.90322 350 The Bell
Children 18:3 16 178.52036 350 Mock The Music
Chris Cagle 17 232.85506 350 Miss Me Baby
John Waite 18 269.76608 350 Missing You
Basshunter 19 223.32036 350 Walk On Water
Jay-Z / Lil Wayne 20 236.01587 350 Hello Brooklyn 2.0
Snow Patrol 21 273.6322 350 It's Beginning To Get To Me
Coldcut 22 203.07546 350 Autumn Leaves
Magic Dirt 23 251.79383 350 Plastic Loveless Letter
J. Karjalainen & Mustat Lasit 24 336.74404 350 Sinisten tÃÂ¤htien alla
OneRepublic 25 224.67873 350 Secrets
Nirvana 26 219.08853 350 Come As You Are
Joyce Cooling 27 248.11057 350 It's Time I Go (Jazz)
Beastie Boys 28 211.722 350 Unite (2009 Digital Remaster)
Usher Featurin

### Validate our Data Model using a SELECT

In [11]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query4 = 'SELECT * FROM my_music_library WHERE session_number = 338 and item_in_session_number = 4'

try:
    rows = session.execute(query4)
   
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name,row.item_in_session_number, row.length_of_song, row.session_number ,row.song_title )

Faithless 4 495.3073 338 Music Matters (Mark Knight Dub)


In [12]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query5 = 'drop table SoundCloud.my_music_library;'
query6 = 'SELECT * from my_music_library'

try:
    session.execute(query5)
    rows = session.execute(query6)
   
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name,row.item_in_session_number, row.length_of_song, row.session_number ,row.song_title )
    

Error from server: code=2200 [Invalid query] message="unconfigured table my_music_library"


In [13]:
cluster.metadata.keyspaces['soundcloud'].tables

{}

## Query2 Table2: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key 

In [14]:
## Task: Query 2: Find the artist_name, song_title (sorted by item_in_session_number) and 
## name(fname and lname) of the user for user_id = 10, session_number = 182 
## make use of create table command                   

### Let's insert our data into of table

In [15]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#

query1 = "CREATE TABLE IF NOT EXISTS my_artist_library(song_title text,fname text,lname text, artist_name text, user_id int,session_number int,item_in_session_number int,PRIMARY KEY(session_number,item_in_session_number,fname,lname)) WITH CLUSTERING ORDER BY(item_in_session_number ASC,fname ASC,lname ASC)";
try:
    session.execute(query1)
except Exception as e:
    print(e)


In [16]:
file_name = 'event_data.csv'


## is we have to do any canacatenation

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) 
    for row in csv_reader:
        query2 = "INSERT INTO my_artist_library(song_title, fname, lname, artist_name, user_id, session_number,item_in_session_number)"
        query2 = query2 + " VALUES (%s, %s, %s, %s, %s,%s,%s)"
        session.execute(query2,(row[9],row[1],row[4],row[0],int(row[10]),int(row[8]),int(row[3]))) 
print("Data Imported from a CSV to NoSQL DB")

Data Imported from a CSV to NoSQL DB


In [17]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query3 = 'SELECT * from my_artist_library'

try:
    rows = session.execute(query3)
   
except Exception as e:
    print(e)
        
for row in rows:
    print (row.artist_name, row.fname , row.item_in_session_number, row.lname,row.session_number, row.song_title, row.user_id,  )

Regina Spektor Layla 0 Griffin 23 The Calculation (Album Version) 24
Octopus Project Layla 1 Griffin 23 All Of The Champs That Ever Lived 24
Tegan And Sara Layla 2 Griffin 23 So Jealous 24
Dragonette Layla 3 Griffin 23 Okay Dolores 24
Lil Wayne / Eminem Layla 4 Griffin 23 Drop The World 24
Soulja Boy Tell'em Layla 5 Griffin 23 Let Me Get Em 24
Bodo Wartke Layla 6 Griffin 23 Liebeslied (Sprachen: Deutsch_ Englisch_ FranzÃÂ¶sisch_ Italienisch_ Spanisch_ HollÃÂ¤ndisch_ Japanisch_ Russisch_ Griechisch_ Klingonisch_ Hessisch) 24
Evanescence Layla 7 Griffin 23 Bring Me To Life 24
Van Halen Layla 8 Griffin 23 Good Enough 24
The Academy Is... Layla 9 Griffin 23 Paper Chase (Album Version) 24
Dwight Yoakam Layla 10 Griffin 23 You're The One 24
The Far East Movement featuring Wiz Khalifa and Bionik Layla 11 Griffin 23 Lowridin 24
Amon Amarth Layla 12 Griffin 23 The Hero 24
Modest Mouse Layla 13 Griffin 23 The Good Times Are Killing Me 24
Bad Company Layla 14 Griffin 23 Morning Sun 24
Bodyrox L

La Vela Puerca Chloe 13 Cuevas 849 La Sin RazÃÂ³n 49
Joshua Radin Chloe 14 Cuevas 849 No Envy No Fear 49
Bowling For Soup Chloe 15 Cuevas 849 Punk Rock 101 49
Paramore Chloe 16 Cuevas 849 The Only Exception (Album Version) 49
Just Jack Chloe 17 Cuevas 849 Starz In Their Eyes 49
Imogen Heap Chloe 19 Cuevas 849 Hide & Seek 49
Ray LaMontagne Chloe 20 Cuevas 849 Shelter 49
Phil Wickham Chloe 21 Cuevas 849 You're Beautiful 49
Justin Timberlake duet with Beyonce Chloe 22 Cuevas 849 Until The End Of Time 49
HervÃÂ© & Kissy Sellout Chloe 23 Cuevas 849 Rikkalicious  49
DeBarge Chloe 24 Cuevas 849 Rhythm Of The Night 49
Jane Wiedlin Chloe 25 Cuevas 849 Blue Kiss 49
Lange Chloe 26 Cuevas 849 Red October 49
System of a Down Chloe 27 Cuevas 849 Revenga 49
Weird Al Yankovic Chloe 28 Cuevas 849 Phony Calls (Parody of "Waterfalls" by TLC) 49
Anberlin Chloe 29 Cuevas 849 The Feel Good Drag 49
Skillet Chloe 30 Cuevas 849 Whispers In The Dark (Radio Edit) 49
Modest Mouse Chloe 31 Cuevas 849 Heart Cooks

Calle 13 Jacob 18 Klein 255 Pi-di-di-di 73
Mae Jacob 19 Klein 255 We're So Far Away 73
Helloween Jacob 20 Klein 255 Where The Rain Grows 73
Justin Timberlake Jacob 21 Klein 255 SeÃÂ±orita 73
Me First And The Gimme Gimmes Jacob 22 Klein 255 Good Bye Earl 73
Usher Jacob 23 Klein 255 Hey Daddy (Daddy's Home) 73
John Mayer Jacob 24 Klein 255 Waiting On The World To Change 73
The Cute Lepers Jacob 25 Klein 255 Prove It 73
Nando Reis Jacob 26 Klein 255 RelicÃÂ¡rio 73
Kid Cudi / Ratatat Jacob 27 Klein 255 Alive (nightmare) 73
Huey Lewis & The News Jacob 28 Klein 255 I Want A New Drug 73
Randy Houser Jacob 29 Klein 255 Boots On 73
Bo Hansson Jacob 30 Klein 255 Lothlorien (2002 Digital Remaster) 73
Karen Clark Sheard Jacob 31 Klein 255 Good 73
Alejandro Sanz Jacob 32 Klein 255 Sin que se note 73
The New Pornographers Jacob 35 Klein 255 Go Places 73
Usher Jacob 36 Klein 255 Hey Daddy (Daddy's Home) 73
Kid Cudi / MGMT / Ratatat Jacob 37 Klein 255 Pursuit Of Happiness (nightmare) 73
The Dust Bro

The Dresden Dolls Avery 74 Watkins 324 Coin-Operated Boy 30
Danger Doom Avery 75 Watkins 324 No Names (Black Debbie) (Album Version) 30
Vilma Palma e Vampiros Avery 76 Watkins 324 La pachanga 30
Little People Avery 77 Watkins 324 Above The Clouds 30
Amy Winehouse Avery 78 Watkins 324 He Can Only Hold Her 30
Alesana Avery 79 Watkins 324 Congratulations_ I Hate You 30
Gazpacho Avery 80 Watkins 324 Premonicion 30
Arcade Fire Avery 81 Watkins 324 Neon Bible 30
Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner Avery 82 Watkins 324 Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile) 30
Crystal Castles Avery 83 Watkins 324 Empathy 30
Angus & Julia Stone Avery 86 Watkins 324 Just A Boy 30
Julian Casablancas Avery 87 Watkins 324 4 Chords of the Apocalypse 30
Usher Featuring Lil' Jon & Ludacris Avery 88 Watkins 324 Yeah! 30
Fergie Avery 89 Watkins 324 Losing My Ground 30
Two Door Cinema Club Avery 90 Watkins 324 Undercover Martyn 30
Whitney Houston Duet With 

Lissi Dancefloor Disaster Kate 84 Harrell 605 I'M Gonna Keep On Driving My Car 97
India.Arie Kate 85 Harrell 605 Interested 97
Linkin Park Kate 86 Harrell 605 Points Of Authority (Album Version) 97
Weezer Kate 87 Harrell 605 Island In The Sun 97
Billy Idol Kate 89 Harrell 605 Flesh For Fantasy 97
Michael BublÃÂ© Kate 90 Harrell 605 Can't Help Falling In Love (Album Version) (Studio) 97
Roger Cicero Ryan 0 Smith 719 Frauen regier'n die Welt 26
Lady GaGa / Colby O'Donis Ryan 1 Smith 719 Just Dance 26
Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner Theodore 0 Harris 340 Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile) 14
Vangelis Jacob 0 Klein 913 Wait For Me 73
James Blake Ryan 0 Smith 706 Footnotes 26
R. Kelly Jordan 0 Hicks 935 Your Body's Callin' 37
C.J. Lewis Jordan 1 Hicks 935 Sweets For My Sweet 37
Eve 6 Kevin 1 Arellano 501 Here's To The Night 66
Five Iron Frenzy Kevin 2 Arellano 501 Canada 66
Evanescence Kevin 3 Arellano 501 My Immortal 

...

Tim Hughes Layla 1 Griffin 672 God of Justice 24
David Cassidy & The Partridge Family Layla 2 Griffin 672 I'll Meet You Halfway 24
Snow Patrol Layla 3 Griffin 672 Crack The Shutters 24
Third Eye Blind Layla 4 Griffin 672 Darwin 24
HYPOCRISY Layla 5 Griffin 672 The Quest 24
Peccatum Layla 6 Griffin 672 And Pray For Me 24
Justyna Steczkowska Layla 7 Griffin 672 Noc... 24
Bloc Party Layla 8 Griffin 672 Your Visits Are Getting Shorter 24
Temprees Layla 9 Griffin 672 At Last 24
Mickie Krause Layla 10 Griffin 672 Orange TrÃÂ¤gt Nur Die MÃÂ¼llabfuhr (Go West) 24
Foolish Things Layla 11 Griffin 672 Who Can Compare 24
Bersuit Vergarabat Layla 12 Griffin 672 Hecho En Buenos Aires 24
ManÃÂ¡ Layla 13 Griffin 672 Mariposa traicionera 24
Newton Faulkner Layla 14 Griffin 672 Feels Like Home 24
BjÃÂ¶rk Layla 15 Griffin 672 Come To Me 24
Olivia Newton-John Layla 16 Griffin 672 Hochmah (Interlude) 24
David Byrne & Fatboy Slim feat. Charmaine Clamor Layla 17 Griffin 672 Walk Like A Woman (Album Versi

Dwight Yoakam Rylan 30 George 1076 You're The One 16
Trouble Rylan 31 George 1076 Tuesday's Child 16
OneRepublic Rylan 33 George 1076 All The Right Moves 16
Switchblade Symphony Rylan 34 George 1076 Wicked 16
Sharooz Rylan 35 George 1076 My Distressor 40Khz ReRub Mix 16
Explosions In The Sky Rylan 36 George 1076 Six Days At The Bottom Of The Ocean 16
Radiohead Rylan 37 George 1076 Everything In Its Right Place 16
Metallica Rylan 38 George 1076 Sabbra Cadabra 16
Eric B. & Rakim Rylan 39 George 1076 As The Rhyme Goes On 16
Regina Spektor Rylan 40 George 1076 Daniel Cowman 16
DJ Dizzy Rylan 41 George 1076 Sexy Bitch 16
Base Ball Bear Rylan 42 George 1076 Sayonara-Nostalgia 16
Basshunter Feat. DJ Mental Theos Bazzheadz Rylan 43 George 1076 Now You're Gone 16
Rihanna Rylan 44 George 1076 G4L 16
Evanescence Rylan 45 George 1076 Hello 16
Mischa Daniels Rylan 46 George 1076 Another Place 16
Vilma Palma e Vampiros Rylan 47 George 1076 Un Camino Hasta Vos 16
Kid Cudi / MGMT / Ratatat Rylan 48 G

Regina Spektor Aleena 25 Kirby 196 Another Town 44
Deborah Harry & Iggy Pop Aleena 26 Kirby 196 Well Did You Evah! 44
Dr. Dre / Eminem / Alvin Joiner Aleena 27 Kirby 196 What's The Difference 44
Marcel Woods Aleena 28 Kirby 196 Advanced (Ton T.B. Mix) 44
OneRepublic Aleena 29 Kirby 196 Apologize 44
Beastie Boys Aleena 30 Kirby 196 The Maestro 44
Paramore Aleena 31 Kirby 196 The Only Exception (Album Version) 44
Varios Aleena 32 Kirby 196 Hasta siempre 44
The Black Keys Aleena 33 Kirby 196 Howlin For You 44
Muse Aleena 34 Kirby 196 Feeling Good 44
Frankie J Aleena 35 Kirby 196 Daddy's Little Girl 44
Kirk Franklin & The Family Aleena 36 Kirby 196 Now Behold The Lamb 44
Cooly's Hot Box Aleena 37 Kirby 196 Make Me Happy 44
Sparks Aleena 38 Kirby 196 Lost And Found 44
Regina Spektor Aleena 39 Kirby 196 The Calculation (Album Version) 44
Subhumans Aleena 40 Kirby 196 Rain 44
Sia Aleena 43 Kirby 196 Beautiful Calm Driving 44
Shadows Fall Aleena 44 Kirby 196 Lead Me Home 44
Eminem / Dr. Dre /

Andrew Bird Chloe 3 Cuevas 1114 Heretics 49
Lonnie Gordon Chloe 4 Cuevas 1114 Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 49
Dropkick Murphys Chloe 5 Cuevas 1114 Shattered 49
Dierks Bentley Chloe 6 Cuevas 1114 Settle For A Slowdown 49
Angelspit Chloe 7 Cuevas 1114 Make You Sin 49
Young Gunz / Beanie Sigel Chloe 8 Cuevas 1114 Roc U 49
Morcheeba Chloe 9 Cuevas 1114 In The Hands Of The Gods (Featuring Biz Markie) 49
Bitter:Sweet Chloe 10 Cuevas 1114 Don't Forget To Breathe 49
BjÃÂ¶rk Chloe 11 Cuevas 1114 Undo 49
Mr. Vegas Chloe 12 Cuevas 1114 Tamale 49
The Menzingers Chloe 13 Cuevas 1114 Straight To Hell 49
Justin Timberlake Chloe 14 Cuevas 1114 Cry Me A River 49
CCCP - Fedeli Alla Linea Chloe 15 Cuevas 1114 Sura (2008 Digital Remaster) 49
Drake / Kanye West / Lil Wayne / Eminem Chloe 16 Cuevas 1114 Forever 49
Jorge Gonzalez Chloe 17 Cuevas 1114 Esta Es Para Hacerte FÃÂ©liz 49
Juanes Chloe 18 Cuevas 1114 La Vida Es Un Ratico 49
Sarah McLachlan Chloe 19 Cuevas 1114 Possession 49

Lady GaGa Aleena 5 Kirby 474 Alejandro 44
Nick Jonas & The Administration Aleena 6 Kirby 474 Who I Am 44
Joshua Radin Aleena 7 Kirby 474 The Fear You Won't Fall 44
Kings Of Leon Aleena 8 Kirby 474 Revelry 44
Spinal Tap Aleena 9 Kirby 474 Rock 'n' Roll Creation 44
Aiden Aleena 10 Kirby 474 Breathless (Album Version) 44
Wyclef Jean Aleena 11 Kirby 474 Something About Mary 44
Matthew Good Aleena 12 Kirby 474 She's In It for the Money 44
Justice Aleena 13 Kirby 474 DVNO 44
Guns N' Roses Aleena 14 Kirby 474 Don't Cry (Original) 44
Justin Bieber / Usher Aleena 15 Kirby 474 First Dance 44
Subhumans Aleena 16 Kirby 474 Rain 44
George Michael Ava 0 Robinson 422 Careless Whisper 13
Brad Paisley With Andy Griffith Sylvie 0 Cruz 681 Waitin' On A Woman 10
King ChangÃÂ³ Cecilia 0 Owens 763 ConfesiÃÂ³n 6
Gang Of Four Cecilia 1 Owens 763 I Found That Essence Rare 6
Line Renaud Cecilia 2 Owens 763 Le Soir 6
The Killers Alivia 1 Terrell 3 Read My Mind 4
Enrique Iglesias Stefany 0 White 867 Escape 83
M

### Validate our Data Model using a SELECT

In [18]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results 
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
## name(fname and lname) of the user for user_id = 10, session_number = 182 

query4 = 'SELECT * FROM my_artist_library WHERE user_id = 10 and session_number = 182 ALLOW FILTERING;'
try:
    rows = session.execute(query4)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.fname , row.lname, row.item_in_session_number, row.session_number, row.song_title, row.user_id,  )
    
print("\n\n\n")    
query5 = 'drop table SoundCloud.my_artist_library;'
query6 = 'SELECT * from my_artist_library'

try:
    session.execute(query5)
    rows = session.execute(query6)
    print("The Table my_artist_library has been deleted from the SoundCloud Keyspace")
   
except Exception as e:
    print(e)

Down To The Bone Sylvie Cruz 0 182 Keep On Keepin' On 10
Three Drives Sylvie Cruz 1 182 Greece 2000 10
Sebastien Tellier Sylvie Cruz 2 182 Kilometer 10
Lonnie Gordon Sylvie Cruz 3 182 Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 10




Error from server: code=2200 [Invalid query] message="unconfigured table my_artist_library"


### Query3 Table3: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [19]:
## Task: Query 3: Find every name(first and lastname) of the user from the SoundCloud app history that listened 
## to the song_title 'All Hands Against His Own'
## make use of create table command                   

In [20]:
query1 = "CREATE TABLE IF NOT EXISTS my_song_library (song_title text,fname text,lname text,PRIMARY KEY(song_title,fname,lname))"

try:
                 session.execute(query1)
except Exception as e:
    print(e)

### Let's insert our data into of table

In [21]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
       
        query2 = "INSERT INTO my_song_library(song_title, fname, lname)"
        query2 = query2 + " VALUES (%s, %s, %s)"
        session.execute(query2,(row[9],row[1],row[4]))

### Validate our Data Model using a SELECT

In [22]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query = "SELECT * FROM my_song_library WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.fname ,row.lname)


Jacqueline Lynch
Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [23]:
cluster.metadata.keyspaces['soundcloud'].tables
print("All tables in the Keyspace -- SoundCLoud are dropped")
print("Tables in the SoundCloud are cleared")

All tables in the Keyspace -- SoundCLoud are dropped
Tables in the SoundCloud are cleared


In [24]:
query = "drop table my_song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = """
   DROP KEYSPACE soundcloud
    """ 
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [25]:
print("All tables in the Ksyspace -- SoundCLoud are dropped")
print("Tables in the SoundCloud are cleared")
print("Updated Keyspaces are : ")

cluster.metadata.keyspaces

All tables in the Ksyspace -- SoundCLoud are dropped
Tables in the SoundCloud are cleared
Updated Keyspaces are : 


{'vehicle_tracker': <cassandra.metadata.KeyspaceMetadata at 0x2a1028840d0>,
 'system_auth': <cassandra.metadata.KeyspaceMetadata at 0x2a102884100>,
 'system_schema': <cassandra.metadata.KeyspaceMetadata at 0x2a1028842e0>,
 'home_secuirty': <cassandra.metadata.KeyspaceMetadata at 0x2a102884a60>,
 'nosqldb': <cassandra.metadata.KeyspaceMetadata at 0x2a1028aee30>,
 'system_distributed': <cassandra.metadata.KeyspaceMetadata at 0x2a1028af400>,
 'system': <cassandra.metadata.KeyspaceMetadata at 0x2a1028af7c0>,
 'system_traces': <cassandra.metadata.KeyspaceMetadata at 0x2a1028bc580>}

### Close the session and cluster connection¶

In [26]:
session.shutdown()
cluster.shutdown()

print("1. Session Closed")
print("2. Cluster Closed")

1. Session Closed
2. Cluster Closed
